<a href="https://colab.research.google.com/github/nickLin1225/Final-Psy-LineBot/blob/main/final_psy_LineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install line-bot-sdk flask
!pip install pyngrok
!ngrok authtoken 填token
!git clone https://github.com/THUDM/ChatGLM2-6B

In [ ]:
# 添加peft、opencc
!pip install -r /content/ChatGLM2-6B/requirements.txt

In [ ]:
# 從本地上傳微調壓縮檔並執行解壓縮
!unzip chatGLM2_6B_QLoRA_t16_v5-20230729T023644Z-001.zip

In [ ]:
# 建置模型
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
base_model_path = "THUDM/chatglm2-6b"
peft_model_path = "/content/chatGLM2_6B_QLoRA_t16_v5/checkpoint-1700"

config = PeftConfig.from_pretrained(peft_model_path)


tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModel.from_pretrained(base_model_path,
                      trust_remote_code=True,
                      device='cuda')

model = PeftModel.from_pretrained(base_model, peft_model_path)

In [ ]:
from flask import Flask, request, abort
from linebot.models.events import FollowEvent
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError, LineBotApiError
from linebot.models import *
import requests
from pyngrok import ngrok
import json
import opencc
import csv

# 建置主程序
app = Flask(__name__)
# Channel access token
line_bot_api = LineBotApi("Channel access token")
# Channel secret
handler = WebhookHandler("Channel secret")


# 啟動server對外接口，使line能夠丟消息進來
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
回應用戶，歡迎用的文字消息

'''

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('哈囉，歡迎來到心理聊天室，有什麼問題都可以跟我說呦！')]
    )

In [ ]:
'''
爬取心理文章，以多頁訊息方式呈現
執行ChatGLM模型
'''
from bs4 import BeautifulSoup
import random

# 要爬取的網頁 URL
url = "https://www.iiispace.com/blog/"

# 發送 HTTP GET 請求並獲取網頁內容
response = requests.get(url)
html_content = response.text

# 使用 BeautifulSoup 解析 HTML 內容
soup = BeautifulSoup(html_content, "html.parser")

# 找到所有文章標題和連結
articles = soup.find_all("h2")
data = []

# 爬取20篇文章資訊
for article in articles[:20]:
    title = article.a.text
    link = article.a["href"]
    article_response = requests.get(link)
    article_html = article_response.text
    article_soup = BeautifulSoup(article_html, "html.parser")
    image = article_soup.find("img", class_="enigma_img_responsive wp-post-image")
    image_url = image["src"] if image else "沒有圖片"
    data.append((title, link, image_url))

# 隨機選取 5 筆文章
random_sample = random.sample(data, 5)
carousel_columns = []

# 建立 CarouselColumns
for title, link, image_url in random_sample:
    carousel_column = CarouselColumn(
        thumbnail_image_url=image_url,
        title="近期的心理文章",
        text=title,
        actions=[
            URIAction(label="詳細內容", uri=link)
        ]
    )
    carousel_columns.append(carousel_column)

carousel_template = CarouselTemplate(columns=carousel_columns)

# 判斷message是"近期的心理文章"或是"清空對話紀錄",執行相對應的函式，都不是的話執行chat_with_llm函式
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    reply_token = event.reply_token
    if event.message.text == '近期的心理文章':
        get_psy_info(event)
    elif event.message.text == '清空對話紀錄':
        clear_chat_history(event)
    else:
        response = chat_with_llm(event.source.user_id, event.message.text)
        line_bot_api.reply_message(event.reply_token, messages=[TextSendMessage(response)])

# 近期的心理文章，多頁訊息方式呈現
def get_psy_info(event):
    random_sample = random.sample(data, 5)
    carousel_columns = []

    for title, link, image_url in random_sample:
        carousel_column = CarouselColumn(
            thumbnail_image_url=image_url,
            title="近期的心理文章",
            text=title,
            actions=[
                URIAction(label="詳細內容", uri=link)
            ]
        )
        carousel_columns.append(carousel_column)

    carousel_template = CarouselTemplate(columns=carousel_columns)

    line_bot_api.reply_message(
        event.reply_token,
        TemplateSendMessage(alt_text="心理相關諮訊", template=carousel_template)
    )

# 清空對話紀錄
def clear_chat_history(event):

    # 在這個函式中，加入清空對話紀錄的程式碼

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage("對話紀錄已清空")
    )

def chat_with_llm(user_id, input_text):
    user_profile = line_bot_api.get_profile(user_id)
    prmpts = "请假装你是一名专业的心理咨商师，根据下列发问者所提供之问题描述一步步进行思考，并以第一人称口吻给出答复。\n问题描述："

    cc = opencc.OpenCC('tw2s')
    converted_input = cc.convert(input_text)

    cc = opencc.OpenCC('s2tw')
    response, history = model.chat(tokenizer, prmpts + converted_input, history=[], repetition_penalty=1.2)
    response = '\n'.join(response.replace(". ", ".").split()).replace(",", "，").replace(".", ". ")
    converted_response = cc.convert(response)
    field_names = ["user_id", "display_name", "picture_url", "input_text", "response"]

    # 將用戶資料以及聊天紀錄存至users.csv
    with open("/content/drive/MyDrive/users.csv", "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)

        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow({
            "user_id": user_profile.user_id,
            "display_name": user_profile.display_name,
            "picture_url": user_profile.picture_url,
            "input_text": input_text,
            "response": converted_response
        })
    return converted_response

In [ ]:
"""
創建 Rich Menu圖文選單
"""

rich_menu_to_create = RichMenu(
    size=RichMenuSize(width=2500, height=843),
    selected=True,
    name="My Rich Menu",
    chat_bar_text="查看更多資訊",
    areas=[
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=0, width=833, height=843),
            action=URIAction(label='Button 1', uri='https://forms.gle/7jbBMRh1CaphF8hv7')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=833, y=0, width=833, height=843),
            action=MessageAction(label='Button 2', text='清空對話紀錄')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=1666, y=0, width=833, height=843),
            action=MessageAction(label='Button 3', text='近期的心理文章')
        )
    ]
)

# 上傳 Rich Menu
try:
    rich_menu_id = line_bot_api.create_rich_menu(rich_menu=rich_menu_to_create)
    print("Rich Menu created. ID:", rich_menu_id)
except LineBotApiError as e:
    print("Failed to create Rich Menu:", e)

# 上傳 Rich Menu 的圖片
image_url = 'https://i.imgur.com/eHvGozp.jpg'
image_content = requests.get(image_url).content
line_bot_api.set_rich_menu_image(rich_menu_id, 'image/png', image_content)

# 設定預設 Rich Menu
line_bot_api.set_default_rich_menu(rich_menu_id)

# 獲取現有的 Rich Menu 列表
# rich_menu_list = line_bot_api.get_rich_menu_list()

# 刪除現有的 Rich Menu
# for rich_menu in rich_menu_list:
#     line_bot_api.delete_rich_menu(rich_menu.rich_menu_id)

In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()